In [2]:
import keras
import numpy as np
import cv2
import os
#from pillow import Image
import pillow
#from PIL.Image import Image
from pathlib import Path
from keras.models import Sequential
#from keras.preprocessing.image import ImageDataGenerator

ModuleNotFoundError: No module named 'pillow'

In [ ]:
size = 512
channels = 3
dataset_path = "images/"
train_path = "/"
validate_path = "/"
test_path = "/"

In [ ]:
# Create directory
def create_dir(folder_name):
    folder = os.path.join(folder_name)
    if not os.path.exists(folder):
        os.mkdir(folder)
    return folder
                          
def save_path(dir_path, file, appended_name):
    file_base = os.path.basename(file)
    file_name, file_ext = os.path.splitext(file_base)   
    return dir_path + file_name + appended_name + file_ext
            
# Create and save black and white images
def convert_RGB2BW(dataset_path):
    bw_path = create_dir("bw_images/")
    grayscale_path = create_dir("gray_images/")
    for file in os.listdir(dataset_path):
        image = Image.open(os.path.join(dataset_path, file))
        
        # Convert image to grayscale based on Lighting
        grayscale = Image.convert('L')
        gray_image = Image.fromarray(grayscale)
        gray_image.save(save_path(gray_path, file, "_GR"))
        
        # Turn grayscale into np array
        blackwhite = np.asarray(grayscale).copy()
        
        # Turn darker pixels black (0) and lighter pixels white (255)
        blackwhite[blackwhite < 128] = 0
        blackwhite[blackwhite >= 128] = 255
        bw_image = Image.fromarray(blackwhite)
        bw_image.save(save_path(bw_path, file, "_BW"))
        
# Preprocess images: creates a white image and pastes data into white image
def square_images(dataset_path, size, background_color=(255, 255, 255, 0)):
    dir_path = create_dir("squared_images/")
    for file in os.listdir(dataset_path):
        image = Image.open(file)
        width, height = image.size
        if(width != height and width != size):
            squared_image = Image.new("1", (size, size), background_color)
            squared_image.paste(image)
            squared_image.save(save_path(dir_path, file, "_SQ"))
                          

In [ ]:
# Image Preprocessing
convert_RGB2BW(dataset_path)

# Load images into array
img_arr = []
for image in os.listdir(bw_images):
    img_read = cv2.imread(images)
    img_arr.append(np.asarray(img_read))

# Convert image array into numpy array
img_np = np.array(img_arr, dtype=float) 

In [ ]:
autoencoder = Sequential()
autoencoder.add(Conv2D(16, (3,3), activation="relu", input_shape(size, size, 1)))
autoencoder.add(MaxPooling2D((2,2), padding = "same"))
autoencoder.add(Conv2D(32, (3,3) activation="relu"))


In [ ]:
autoencoder.fit(img_np, validation_split=0.1, batch_size=x, epoch=x, shuffle=True)